### Reproduce cookbook

https://openmm.github.io/openmm-cookbook/latest/notebooks/cookbook/Computing%20Interaction%20Energies.html

In [1]:
from openmm import *
from openmm.app import *
from openmm.unit import *

In [2]:
pdb = PDBFile('villin.pdb')
solvent = set([a.index for a in pdb.topology.atoms() if a.residue.name in ('HOH', 'Cl')])
protein = set([a.index for a in pdb.topology.atoms() if a.index not in solvent])

In [3]:
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME)
for force in system.getForces():
    if isinstance(force, NonbondedForce):
        force.setForceGroup(0)
        force.addGlobalParameter("solute_coulomb_scale", 1)
        force.addGlobalParameter("solute_lj_scale", 1)
        force.addGlobalParameter("solvent_coulomb_scale", 1)
        force.addGlobalParameter("solvent_lj_scale", 1)
        for i in range(force.getNumParticles()):
            charge, sigma, epsilon = force.getParticleParameters(i)
            force.setParticleParameters(i, 0, 0, 0)
            if i in protein:
                force.addParticleParameterOffset("solute_coulomb_scale", i, charge, 0, 0)
                force.addParticleParameterOffset("solute_lj_scale", i, 0, sigma, epsilon)
            else:
                force.addParticleParameterOffset("solvent_coulomb_scale", i, charge, 0, 0)
                force.addParticleParameterOffset("solvent_lj_scale", i, 0, sigma, epsilon)
        for i in range(force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = force.getExceptionParameters(i)
            force.setExceptionParameters(i, p1, p2, 0, 0, 0)
    else:
        force.setForceGroup(2)

In [4]:
integrator = VerletIntegrator(0.001*picosecond)
context = Context(system, integrator)
context.setPositions(pdb.positions)

def energy(solute_coulomb_scale, solute_lj_scale, solvent_coulomb_scale, solvent_lj_scale):
    context.setParameter("solute_coulomb_scale", solute_coulomb_scale)
    context.setParameter("solute_lj_scale", solute_lj_scale)
    context.setParameter("solvent_coulomb_scale", solvent_coulomb_scale)
    context.setParameter("solvent_lj_scale", solvent_lj_scale)
    return context.getState(getEnergy=True, groups={0}).getPotentialEnergy()

total_coulomb = energy(1, 0, 1, 0)
solute_coulomb = energy(1, 0, 0, 0)
solvent_coulomb = energy(0, 0, 1, 0)
total_lj = energy(0, 1, 0, 1)
solute_lj = energy(0, 1, 0, 0)
solvent_lj = energy(0, 0, 0, 1)
print('Coulomb interaction energy:', total_coulomb - solute_coulomb - solvent_coulomb)
print('LJ interaction energy:', total_lj - solute_lj - solvent_lj)

Coulomb interaction energy: -5638.628673112849 kJ/mol
LJ interaction energy: 220.6627468027218 kJ/mol


#### Test with current data

In [5]:
del pdb, solvent, protein, forcefield, system, force, integrator, context

In [6]:
from openff.toolkit import ForceField, Molecule, Topology
from openmm import LangevinMiddleIntegrator, unit
from openff.interchange import Interchange
import mdtraj

In [7]:
integrator = LangevinMiddleIntegrator(300, 1, 4)

In [8]:
print("Building ligand..")
ligand = Molecule.from_file("final.sdf", allow_undefined_stereo=True)

Building ligand..


In [9]:
ligand_intrcg = Interchange.from_smirnoff(force_field=ForceField("openff_unconstrained-2.0.0.offxml"), topology=[ligand],)

In [10]:
print("\nBuilding protein..")
protein = Topology.from_pdb("final.pdb") # apo protein + ions


Building protein..


In [11]:
protein_intrcg = Interchange.from_smirnoff(
    force_field=ForceField("ff14sb_off_impropers_0.0.3.offxml", "openff_unconstrained-2.0.0.offxml"),
    topology=protein,
)

In [12]:
%env INTERCHANGE_EXPERIMENTAL=1

env: INTERCHANGE_EXPERIMENTAL=1


In [13]:
print("\nBuilding complex..")
complex_intrcg = protein_intrcg + ligand_intrcg


Building complex..


/lila/home/takabak/mambaforge/envs/espfit/lib/python3.11/site-packages/openff/interchange/_experimental.py:35: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  return func(*args, **kwargs)


In [14]:
print("Minimizing..")
simulation = complex_intrcg.to_openmm_simulation(integrator)
start_pot_energy = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole
simulation.minimizeEnergy(maxIterations=3)
end_pot_energy = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

Minimizing..


In [15]:
print(f"{round(start_pot_energy, 2)} -> {round(end_pot_energy, 2)}")

1259.4 -> -3718.85


In [16]:
forces = list(simulation.system.getForces())
_force = forces[0]

In [17]:
# https://github.com/openmm/openmm/blob/abadc8211158900209a0893876e69ba7fe65968c/olla/include/openmm/kernels.h#L581
print(_force.getNonbondedMethod())  # NoCutoff
print(_force.getCutoffDistance())

0
0.9 nm


#### (Vacuum) compute ele and vdw interactions

In [18]:
# check complex topology
res = [ r.name for r in simulation.topology.residues() ]
set(res)

{'ALA',
 'ARG',
 'ASN',
 'ASP',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'ILE',
 'LEU',
 'LYS',
 'MET',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'UNK',
 'VAL'}

In [19]:
protein_atom_indices = set([a.index for a in simulation.topology.atoms() if a.residue.name not in ('UNK')])
ligand_atom_indices = set([a.index for a in simulation.topology.atoms() if a.residue.name in ('UNK')])

In [20]:
forces = list(simulation.system.getForces())
forces

[<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fb43c292af0> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7fb43c291380> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7fb43c293900> >,
 <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7fb43c291e60> >]

In [21]:
import openmm

for force in simulation.system.getForces():
    if isinstance(force, openmm.openmm.NonbondedForce):
        force.setForceGroup(0)
        force.addGlobalParameter("solute_coulomb_scale", 1)
        force.addGlobalParameter("solute_lj_scale", 1)
        force.addGlobalParameter("solvent_coulomb_scale", 1)
        force.addGlobalParameter("solvent_lj_scale", 1)
        for i in range(force.getNumParticles()):
            charge, sigma, epsilon = force.getParticleParameters(i)
            force.setParticleParameters(i, 0, 0, 0)
            if i in protein_atom_indices:
                force.addParticleParameterOffset("solute_coulomb_scale", i, charge, 0, 0)
                force.addParticleParameterOffset("solute_lj_scale", i, 0, sigma, epsilon)
            else:
                force.addParticleParameterOffset("solvent_coulomb_scale", i, charge, 0, 0)
                force.addParticleParameterOffset("solvent_lj_scale", i, 0, sigma, epsilon)
        for i in range(force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = force.getExceptionParameters(i)
            force.setExceptionParameters(i, p1, p2, 0, 0, 0)
    else:
        force.setForceGroup(2)

In [22]:
# don't know why but regenerating the context seems to work
import copy
integrator_copy = copy.deepcopy(integrator)
context = Context(simulation.system, integrator_copy)
context.setPositions(simulation.context.getState(getPositions=True).getPositions())

In [23]:
# this should be equal to 
# end_pot_energy = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole
context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

-30550.49322119234

In [24]:
end_pot_energy

-3718.851883661991

In [25]:
def energy(solute_coulomb_scale, solute_lj_scale, solvent_coulomb_scale, solvent_lj_scale):
    context.setParameter("solute_coulomb_scale", solute_coulomb_scale)
    context.setParameter("solute_lj_scale", solute_lj_scale)
    context.setParameter("solvent_coulomb_scale", solvent_coulomb_scale)
    context.setParameter("solvent_lj_scale", solvent_lj_scale)
    return context.getState(getEnergy=True, groups={0}).getPotentialEnergy() / unit.kilocalories_per_mole

total_coulomb = energy(1, 0, 1, 0)
solute_coulomb = energy(1, 0, 0, 0)
solvent_coulomb = energy(0, 0, 1, 0)
total_lj = energy(0, 1, 0, 1)
solute_lj = energy(0, 1, 0, 0)
solvent_lj = energy(0, 0, 0, 1)
print('Coulomb interaction energy:', total_coulomb - solute_coulomb - solvent_coulomb)
print('LJ interaction energy:', total_lj - solute_lj - solvent_lj)

Coulomb interaction energy: -32.96294492818568
LJ interaction energy: -46.128519253344265


In [26]:
# check nothing changed
context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

17932.92426432705

#### (GBSA?) compute ele and vdw interactions

https://github.com/openforcefield/openff-interchange/blob/e3ad1bf9be646a8074c59b206086b04c84e7bb88/openff/interchange/_tests/interoperability_tests/test_openmm.py#L1170

In [27]:
system_gbsa = Interchange.from_smirnoff(
    force_field=ForceField("ff14sb_off_impropers_0.0.3.offxml", "openff_unconstrained-2.0.0.offxml", 
                           "/home/takabak/mambaforge/envs/espfit/lib/python3.11/site-packages/openff/interchange/_tests/data/gbsa.offxml"),
    topology=complex_intrcg.topology,
)

In [28]:
system_gbsa = system_gbsa.to_openmm()

In [29]:
forces = list(system_gbsa.getForces())
forces

[<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fb3b73dced0> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7fb3b73dea30> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7fb3b73dfea0> >,
 <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7fb3b73dd6e0> >,
 <openmm.openmm.CustomGBForce; proxy of <Swig Object of type 'OpenMM::CustomGBForce *' at 0x7fb3b73dd050> >]

In [30]:
forces[0].getNonbondedMethod()

0

In [35]:
import copy
integrator_copy2 = copy.deepcopy(integrator)
context_gbsa = Context(system_gbsa, integrator_copy2)
context_gbsa.setPositions(simulation.context.getState(getPositions=True).getPositions())

In [36]:
context_gbsa.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

-32845.16499191843

In [37]:
for force in system_gbsa.getForces():
    if isinstance(force, openmm.openmm.NonbondedForce):
        force.setForceGroup(0)
        force.addGlobalParameter("solute_coulomb_scale", 1)
        force.addGlobalParameter("solute_lj_scale", 1)
        force.addGlobalParameter("solvent_coulomb_scale", 1)
        force.addGlobalParameter("solvent_lj_scale", 1)
        for i in range(force.getNumParticles()):
            charge, sigma, epsilon = force.getParticleParameters(i)
            force.setParticleParameters(i, 0, 0, 0)
            if i in protein_atom_indices:
                force.addParticleParameterOffset("solute_coulomb_scale", i, charge, 0, 0)
                force.addParticleParameterOffset("solute_lj_scale", i, 0, sigma, epsilon)
            else:
                force.addParticleParameterOffset("solvent_coulomb_scale", i, charge, 0, 0)
                force.addParticleParameterOffset("solvent_lj_scale", i, 0, sigma, epsilon)
        for i in range(force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = force.getExceptionParameters(i)
            force.setExceptionParameters(i, p1, p2, 0, 0, 0)
    else:
        force.setForceGroup(2)

In [38]:
def energy_gbsa(solute_coulomb_scale, solute_lj_scale, solvent_coulomb_scale, solvent_lj_scale):
    context_gbsa.setParameter("solute_coulomb_scale", solute_coulomb_scale)
    context_gbsa.setParameter("solute_lj_scale", solute_lj_scale)
    context_gbsa.setParameter("solvent_coulomb_scale", solvent_coulomb_scale)
    context_gbsa.setParameter("solvent_lj_scale", solvent_lj_scale)
    return context_gbsa.getState(getEnergy=True, groups={0}).getPotentialEnergy() / unit.kilocalories_per_mole

total_coulomb = energy_gbsa(1, 0, 1, 0)
solute_coulomb = energy_gbsa(1, 0, 0, 0)
solvent_coulomb = energy_gbsa(0, 0, 1, 0)
total_lj = energy_gbsa(0, 1, 0, 1)
solute_lj = energy_gbsa(0, 1, 0, 0)
solvent_lj = energy_gbsa(0, 0, 0, 1)
print('Coulomb interaction energy:', total_coulomb - solute_coulomb - solvent_coulomb)
print('LJ interaction energy:', total_lj - solute_lj - solvent_lj)

Coulomb interaction energy: -32.96294492818568
LJ interaction energy: -46.128519253344265


In [39]:
context_gbsa.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

15638.252493600969

In [40]:
# I need to get the SA energy? -> is this sum of polar and non-polar contributions or just the non-polar contribution?
# http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.GBSAOBCForce.html